In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import differential_evolution

In [2]:
org_df = pd.read_csv('waterDataTraining.csv')
df = org_df.copy()
# df.dropna(inplace = True)
df.head()

,Time,Tp,pH,Cond,Turb,SAC,PFM,Event
0,2017-07-01 00:00:00,6.94,8.60774,0.020954,0.125931,3.58683,43.7559,False
1,2017-07-01 00:01:00,6.93,8.60589,0.020965,0.127219,3.59025,43.4366,False
2,2017-07-01 00:02:00,6.94,8.60220,0.020968,0.126482,3.58318,43.5994,False
3,2017-07-01 00:03:00,6.94,8.60220,0.020972,0.126184,3.58769,43.3704,False
4,2017-07-01 00:04:00,6.94,8.60405,0.020973,0.127908,3.58287,43.1656,False


In [3]:
df['Event'].value_counts()

False    132268
True        212
Name: Event, dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132480 entries, 0 to 132479
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    132480 non-null  object 
 1   Tp      132266 non-null  float64
 2   pH      132272 non-null  float64
 3   Cond    132276 non-null  float64
 4   Turb    132275 non-null  float64
 5   SAC     132280 non-null  float64
 6   PFM     132269 non-null  float64
 7   Event   132480 non-null  bool   
dtypes: bool(1), float64(6), object(1)
memory usage: 7.2+ MB


In [5]:
# df['Tp_m_avg'] 	= df['Tp'].rolling(window = 30).mean()
# df['pH_m_avg'] 	= df['pH'].rolling(window = 30).mean()
# df['Turb_m_avg'] = df['Turb'].rolling(window = 30).mean()
# df['SAC_m_avg'] 	= df['SAC'].rolling(window = 30).mean()
# df['PFM_m_avg'] 	= df['PFM'].rolling(window = 30).mean()
# df['Cond_m_avg'] = df['Cond'].rolling(window = 30).mean()

In [6]:
# df.dropna(inplace = True)
# df = org_df.copy()
# df2 = df.copy()
# df2['Tp'] = df['Tp'].fillna(df['Tp'].median())
# df2['pH'] = df['pH'].fillna(df['pH'].median())
# df2['Cond'] = df['Cond'].fillna(df['Cond'].median())
# df = df.fillna(df.median(numeric_only = True))
# df['Tp_m_avg'] 	= df['Tp'].rolling(window = 30).median()
# df['pH_m_avg'] 	= df['pH'].rolling(window = 30).median()
# df['Turb_m_avg'] = df['Turb'].rolling(window = 30).median()
# df['SAC_m_avg'] 	= df['SAC'].rolling(window = 30).median()
# df['PFM_m_avg'] 	= df['PFM'].rolling(window = 30).median()
# df['Cond_m_avg'] = df['Cond'].rolling(window = 30).median()


# df['Tp_m_min'] 	= df['Tp'].rolling(window = 30).min()
# df['pH_m_min'] 	= df['pH'].rolling(window = 30).min()
# df['Turb_m_min'] = df['Turb'].rolling(window = 30).min()
# df['SAC_m_min'] 	= df['SAC'].rolling(window = 30).min()
# df['PFM_m_min'] 	= df['PFM'].rolling(window = 30).min()
# df['Cond_m_min'] = df['Cond'].rolling(window = 30).min()


# df['Tp_m_mam'] 	= df['Tp'].rolling(window = 30).max()
# df['pH_m_mam'] 	= df['pH'].rolling(window = 30).max()
# df['Turb_m_mam'] = df['Turb'].rolling(window = 30).max()
# df['SAC_m_mam'] 	= df['SAC'].rolling(window = 30).max()
# df['PFM_m_mam'] 	= df['PFM'].rolling(window = 30).max()
# df['Cond_m_mam'] = df['Cond'].rolling(window = 30).max()
df.dropna(inplace = True)

In [7]:
X = df.drop(['Time', 'Event'], axis = 1)
y = df['Event']

In [8]:
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print(" X_train dataset: ", X_train.shape)
print(" y_train dataset: ", y_train.shape)
print(" X_test dataset: ", X_test.shape)
print(" y_test dataset: ", y_test.shape)

 X_train dataset:  (92548, 6)
 y_train dataset:  (92548,)
 X_test dataset:  (39664, 6)
 y_test dataset:  (39664,)


In [9]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [11]:
from sklearn.metrics import f1_score

def get_model_f1(x):
    model = RandomForestClassifier(n_estimators = int(x[0]), random_state = 123)
    model.fit(X_train_res, y_train_res)
    y_pred = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * (precision * recall) / (precision + recall)
    print(f"In Func, 1-f1:{1- f1_score(y_test, y_pred)}, f1:{f1}, c: {x[0]}")
    return 1-f1
    # DE tries to minimize the objective function. I couldn't find a way to change it to be able to maximize the obj func.
    # Hence, I'm returning 1-f1 score so that it minimises 1-f1 -> same as maximising f1

In [12]:
# bounds = [(0, 1e-1)]
bounds = [(100, 1000)]
result = differential_evolution(func = get_model_f1, bounds = bounds, maxiter = 10, popsize = 10)
print(result.x, result.fun)

In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 999.9626228490902
In Func, 1-f1:0.1074380165289257, f1:0.8925619834710743, c: 191.93035144264587
In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 322.14050616187507
In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 830.3156134667139
In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 495.4589534241855
In Func, 1-f1:0.1074380165289257, f1:0.8925619834710743, c: 155.04938434965936
In Func, 1-f1:0.10924369747899165, f1:0.8907563025210083, c: 449.50393216883066
In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 719.9187681957112
In Func, 1-f1:0.10924369747899165, f1:0.8907563025210083, c: 599.0690758403142
In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 736.5641466296141
In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 277.86688828144975
In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 247.12872350890632
In Func, 1-f1:0.1074380165289257, f1:0.892561983

In Func, 1-f1:0.1074380165289257, f1:0.8925619834710743, c: 430.65159232616116
In Func, 1-f1:0.10924369747899165, f1:0.8907563025210083, c: 450.5631268747219
In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 606.2274929663577
In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 273.9018847680729
In Func, 1-f1:0.11666666666666659, f1:0.8833333333333334, c: 272.9680451681536
In Func, 1-f1:0.09999999999999998, f1:0.9, c: 437.9919289793362
In Func, 1-f1:0.09999999999999998, f1:0.9, c: 437.9919289893362
[437.99192898] 0.09999999999999998


In [15]:
# model = RandomForestClassifier(n_estimators = 437, random_state = 123)
# model.fit(X_train_res, y_train_res)
# y_pred = model.predict(X_test)
# # tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
# # recall = tp / (tp + fn)
# # precision = tp / (tp + fp)
# # f1 = 2 * (precision * recall) / (precision + recall)
get_metrics(y_pred, y_test)

(0.9996974586526826, 0.8571428571428571, 0.9473684210526315, 0.9)

In [14]:
from sklearn.metrics import confusion_matrix
def get_metrics(Y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(Y_true, y_pred).ravel()
    acc = (tp + tn) / (tp + fp + tn + fn)
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * (precision * recall) / (precision + recall)
    return acc, precision, recall, f1